In [1]:
%%capture

!pip install -q protobuf==3.20.0
!pip install -q sentence-transformers==3.0.1 scikit-learn==1.5.1 tqdm numpy
!pip install langchain-community

In [2]:
# @title API Request

# @markdown The public API is at https://cqztaifwfa.us-east-1.awsapprunner.com

# @markdown Go [here](https://cqztaifwfa.us-east-1.awsapprunner.com/docs#/Chat%20LLM/get_candidate_recipes_get_candidate_recipes_post) to read the documentation on the route that we are using

import requests
import json




In [3]:
# @markdown Build a recipe candidate request

# @markdown `candidate_recipes` is the total number of candidates that will be returned

# @markdown `preferences` is a free text input where the user will describe generally what they want for meals

# @markdown `queries_per_meal` is the number of individual dense text queries that will be generated by the LLM for each meal (breakfast, lunch, dinner). Increasing this number will give more variety in the returned candidate recipes.

# @markdown `target_calories` daily calorie target

# @markdown `target_protein` daily protein target

# @markdown `target_fat` daily fat target

# @markdown `target_carbs` daily carbs target

# @markdown `exclusions` is a free text input where the user can describe what they don't want in meals

# @markdown `dietary` is a list of dietary flags ("vegetarian", "vegan", "gluten_free", "pescatarian", "gluten_free")

# @markdown `breakfast_targets` A dictionary where you can specify the target macros as a percentage of the daily macros and the tolerance

# @markdown `lunch_targets` A dictionary where you can specify the target macros as a percentage of the daily macros and the tolerance

# @markdown `dinner_targets` A dictionary where you can specify the target macros as a percentage of the daily macros and the tolerance

candidate_recipes = 42 # @param {"type":"number","min":1,"max":100,"step":1}
preferences = "I want mostly egg meals for breakfast, quick and easy lunches, and hearty dinners" # @param {type:"string"}
queries_per_meal = 7 # @param {"type":"number","min":1,"max":10,"step":1}
target_calories = 2500 # @param {"type":"number"}
target_protein = 100 # @param {"type":"number"}
target_fat = 60 # @param {"type":"number"}
target_carbs = 200 # @param {"type":"number"}
exclusions = "peanuts, mushrooms" # @param {type:"string"}
vegetarian = True # @param {"type":"boolean"}
vegan = False # @param {"type":"boolean"}
gluten_free = False # @param {"type":"boolean"}
pescatarian = False # @param {"type":"boolean"}
dairy_free = False # @param {"type":"boolean"}
dietary = []
if vegetarian:
  dietary.append("vegetarian")
if vegan:
  dietary.append("vegan")
if gluten_free:
  dietary.append("gluten-free")
if pescatarian:
  dietary.append("pescatarian")
if dairy_free:
  dietary.append("dairy_free")
breakfast_macro_target_pct = 30 # @param {"type":"number"}
breakfast_macro_tolerance_pct = 10 # @param {"type":"number"}
lunch_macro_target_pct = 30 # @param {"type":"number"}
lunch_macro_tolerance_pct = 10 # @param {"type":"number"}
dinner_macro_target_pct = 40 # @param {"type":"number"}
dinner_macro_tolerance_pct = 10 # @param {"type":"number"}

API_BASE = "https://cqztaifwfa.us-east-1.awsapprunner.com"
API_CANDIDATE_ROUTE = f"{API_BASE}/get-candidate-recipes"

def api_get_candidate_recipes(
    candidate_recipes: int = candidate_recipes,
    preferences: str = preferences,
    queries_per_meal: int = queries_per_meal,
    target_calories: int = target_calories,
    target_protein: int = target_protein,
    target_fat: int = target_fat,
    target_carbs: int = target_carbs,
    exclusions: str = exclusions,
    dietary: list = dietary,
    breakfast_macro_target_pct: int = breakfast_macro_target_pct,
    breakfast_macro_tolerance_pct: int = breakfast_macro_tolerance_pct,
    lunch_macro_target_pct: int = lunch_macro_target_pct,
    lunch_macro_tolerance_pct: int = lunch_macro_tolerance_pct,
    dinner_macro_target_pct: int = dinner_macro_target_pct,
    dinner_macro_tolerance_pct: int = dinner_macro_tolerance_pct
):

  # Make sure we can connect to the API
  api_status_route = f"{API_BASE}/status"
  response = requests.get(api_status_route)
  if response.status_code == 200:
    print("Successfully connected to the API")
    print(json.dumps(response.json(),indent=2))
  else:
    raise ConnectionError(f"Failed to connect to the API. Status code: {response.status_code}")

  req = {
    "candidate_recipes": candidate_recipes,
    "preferences": preferences,
    "queries_per_meal": queries_per_meal,
    "target_calories": target_calories,
    "target_protein": target_protein,
    "target_fat": target_fat,
    "target_carbs": target_carbs,
    "exclusions": exclusions,
    "dietary": dietary,
    "breakfast_targets": {
      "macro_target_pct": breakfast_macro_target_pct,
      "macro_tolerance_pct": breakfast_macro_tolerance_pct
    },
    "lunch_targets": {
      "macro_target_pct": lunch_macro_target_pct,
      "macro_tolerance_pct": lunch_macro_tolerance_pct
    },
    "dinner_targets": {
      "macro_target_pct": dinner_macro_target_pct,
      "macro_tolerance_pct": dinner_macro_tolerance_pct
    },
    "num_days": 7, # Not actually used in the API call
  }

  headers = {
      "Content-Type": "application/json"
  }

  print("Sending post request")
  response = requests.post(API_CANDIDATE_ROUTE, headers=headers, json=req)
  if response.status_code != 200:
    raise Exception(f"API call failed with status code {response.status_code}")
  print("Received valid payload")
  return response.json()



In [4]:
# @title # Test out API call
# @markdown The previous cell can be used to set our default params for the api call, so we can just call with the defaults now.
results = api_get_candidate_recipes()


Successfully connected to the API
{
  "collection": "recipes",
  "client_ok": true,
  "collection_ok": true,
  "sparse_enabled": true,
  "model_loaded": true,
  "tfidf_loaded": false,
  "ready_for_dense": true,
  "ready_for_hybrid": true,
  "error": null
}
Sending post request
Received valid payload


In [5]:
# @markdown Show the top level keys in the response. Should be "candidate_recipes", "original_request", "queries", and "meal_counts"
print(f"Top-Level Keys = {results.keys()}")

Top-Level Keys = dict_keys(['candidate_recipes', 'original_request', 'queries', 'meal_counts'])


In [6]:
# @markdown Show the original request passthrough (we may remove this in the future depending on how we implement in the API)
results.get("original_request")

{'target_calories': 2500.0,
 'target_protein': 100.0,
 'target_fat': 60.0,
 'target_carbs': 200.0,
 'breakfast_targets': {'macro_target_pct': 30.0, 'macro_tolerance_pct': 10.0},
 'lunch_targets': {'macro_target_pct': 30.0, 'macro_tolerance_pct': 10.0},
 'dinner_targets': {'macro_target_pct': 40.0, 'macro_tolerance_pct': 10.0},
 'dietary': ['vegetarian'],
 'num_days': 7,
 'queries_per_meal': 7,
 'candidate_recipes': 42,
 'preferences': 'I want mostly egg meals for breakfast, quick and easy lunches, and hearty dinners',
 'exclusions': 'peanuts, mushrooms'}

In [7]:
# @markdown Show the list of queries used in the vectorDB searches
results.get("queries")

['vegetarian spinach feta egg muffins breakfast',
 'vegetarian caprese salad quick lunch',
 'vegetarian vegetable bean chili hearty dinner',
 'vegetarian avocado egg toast breakfast',
 'vegetarian greek salad pita pockets quick lunch',
 'vegetarian stuffed bell peppers quinoa black beans hearty dinner',
 'vegetarian cheesy egg vegetable breakfast casserole',
 'vegetarian tomato mozzarella panzanella salad quick lunch',
 'vegetarian lentil vegetable curry hearty dinner',
 'vegetarian egg vegetable breakfast burrito',
 'vegetarian quinoa black bean salad quick lunch',
 'vegetarian vegetable lasagna hearty dinner',
 'vegetarian scrambled eggs spinach tomatoes breakfast',
 'vegetarian roasted vegetable hummus wrap quick lunch',
 'vegetarian chickpea spinach stew hearty dinner',
 'vegetarian egg cheese breakfast sandwich',
 'vegetarian mediterranean couscous salad quick lunch',
 'vegetarian vegetable tofu stir-fry hearty dinner',
 'vegetarian baked eggs spinach mushrooms breakfast',
 'veget

In [8]:
# @markdown show the recipe counts (number of breakfast, lunch, and dinner recipes). If the macro tolerance isn't wide enough this might not return an equal number of recipes for each meal.
results.get("meal_counts")

{'breakfast': 14, 'lunch': 14, 'dinner': 14}

In [9]:
# @markdown printing the entire list of recipes is a lot. Let's just print the total number of recipes returned and one of the recipes at random as an example

print(f"Total number of recipes returned = {len(results.get('candidate_recipes'))}")

import random
random_recipe = random.choice(results.get("candidate_recipes"))
print(json.dumps(random_recipe,indent=2))

Total number of recipes returned = 42
{
  "title": "Ww Lentil One Pot Casserole",
  "calories": 823.0126966667,
  "protein_g": 25.3146881219,
  "fat_g": 3.1213516812,
  "carbs_g": 4.3428475594,
  "description": "Hearty, comforting one-pot casserole with lentils, rice, carrots and aromatic spices; easy, satisfying weeknight meal.",
  "instructions": "Rinse lentils.\nPlace all ingredients into a large pot.\nBring to a boil.\nReduce heat, cover, and cook until rice is done, about 20 minutes.",
  "ingredients": [
    "lentils, raw",
    "rice, white, long-grain, regular, unenriched, cooked without salt",
    "carrots, raw",
    "water, bottled, generic",
    "onions, raw",
    "spices, basil, dried",
    "spices, garlic powder",
    "oil, olive, salad or cooking"
  ],
  "quantities": [
    "6 3/4",
    "12",
    "2",
    "3",
    "7",
    "1",
    "1",
    "1"
  ],
  "units": [
    "ounce",
    "cup",
    "cup",
    "cup",
    "g",
    "teaspoon",
    "teaspoon",
    "teaspoon"
  ],
  "mea

# LLM Generation

In [10]:
%%capture
!pip install cohere==4.52 # Install a compatible version

In [11]:
# get user data
from google.colab import userdata

In [12]:
COHERE_API_KEY = userdata.get('COHERE_API_KEY')

In [ ]:
def build_llm_context(
    meal_candidates: list[dict[str,int,float,list]],
    max_context: int = 80
  ):
  """
  meal_candidates: list of python dictionaries representing recipes
  max_context: number of recipes to include in the context
  """
  # Truncate or chunk to fit LLM context window
  max_context = 80  # number of recipes to include
  selected = meal_candidates[:max_context]
  return "\n".join([json.dumps(recipe,indent=2) for recipe in selected])

In [ ]:
build_llm_context(results.get("candidate_recipes"))

'{\n  "title": "Garden Frittata",\n  "calories": 627.45,\n  "protein_g": 46.525,\n  "fat_g": 19.8225,\n  "carbs_g": 35.73,\n  "description": "A versatile, egg-based breakfast dish with vegetables and cheese, baked in a square pan; great for using up leftover ingredients.",\n  "instructions": "Pre heat oven to 400.\\nMix all ingredients in a large mixing bowl.\\nSpray an 8x8-inch baking dish with a nonstick cooking spray and pour all ingredients into the dish.\\nBake for 30-45 minutes depending on your oven.\\nWhen done let the frittata sit for 15 minutes (to set) and then slice and serve.\\nUse your imagination and use different vegetables or whatever you have on hand.",\n  "ingredients": [\n    "egg substitute, powder",\n    "onions, raw",\n    "asparagus, raw",\n    "broccoli, raw",\n    "cheese, cheddar",\n    "mushrooms, white, raw",\n    "salt, table",\n    "spices, pepper, black",\n    "spices, curry powder",\n    "spices, pepper, red or cayenne"\n  ],\n  "quantities": [\n    "2"

In [20]:

from langchain.chat_models import ChatCohere
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
import os

os.environ["COHERE_API_KEY"] = "UoDhu7CVV5W3v5bxuCy6WU69cdxIa4mZGtgnb1wb"
# model = "command-r-plus"
model = "command-a-03-2025"

cohere_chat_model = ChatCohere(
    model=model,
    temperature=0.3,
    max_tokens=int(2**17) # 1024 wasn't enough with all the ingredients, units, and quantities
)

/tmp/ipython-input-1047191708.py:11: LangChainDeprecationWarning: The class `ChatCohere` was deprecated in LangChain 0.0.30 and will be removed in 1.0. An updated version of the class exists in the `langchain-cohere package and should be used instead. To use it run `pip install -U `langchain-cohere` and import as `from `langchain_cohere import ChatCohere``.
  cohere_chat_model = ChatCohere(


In [ ]:
# @title Tell LLM to use daily macros to pick meals

DAILY_KCAL = results.get("original_request").get("target_calories")
DAILY_PROTEIN = results.get("original_request").get("target_protein")
DAILY_FAT = results.get("original_request").get("target_fat")
DAILY_CARBS = results.get("original_request").get("target_carbs")

context = build_llm_context(results.get("candidate_recipes"))

# SOFT_CAL_MIN = DAILY_KCAL * 0.1
# SOFT_CAL_MAX = DAILY_KCAL * 0.5
# SOFT_PROTEIN_MIN = DAILY_PROTEIN * 0.1
# SOFT_PROTEIN_MAX = DAILY_PROTEIN * 0.5
# SOFT_FAT_MIN = DAILY_FAT * 0.1
# SOFT_FAT_MAX = DAILY_FAT * 0.5
# SOFT_CARBS_MIN = DAILY_CARBS * 0.1
# SOFT_CARBS_MAX = DAILY_CARBS * 0.5

In [ ]:

combined_daily_prompt = PromptTemplate(
    input_variables=[
        "context",
        "question",
        "preferences",
        "daily_cal",
        "daily_protein",
        "daily_fat",
        "daily_carbs",
    ],
    template="""
You are a nutrition assistant helping to build a 7-day meal plan.
Each day should have 3 meals: Breakfast, Lunch, and Dinner.

Your goal is to select meals so that the **daily totals** are as close as possible to the targets below:

- Calories: {daily_cal} kcal
- Protein: {daily_protein} g
- Fat: {daily_fat} g
- Carbs: {daily_carbs} g

User preferences to consider (e.g. dislikes, dietary restrictions, flavor preferences):
{preferences}

Here is a list of candidate meals with macros, flavors, and instructions:
---
{context}
---

Question:
{question}

Return the meal plan strictly as valid JSON using this structure (double curly braces used to escape JSON template variables):
{{
  "days": [
    {{
      "day": 1,
      "meals": {{
        "breakfast": {{
          "title": "<Recipe Title>",
          "calories": 0,
          "protein_g": 0,
          "fat_g": 0,
          "carbs_g": 0,
          "ingredients": ["<ingredient 1>", "<ingredient 2>", "..."],
          "units": ["<unit 1>", "<unit 2>", "..."],
          "quantities": ["<quantity 1>", "<quantity 2>", "..."],
          "instructions": "<full instructions text>"
        }},
        "lunch": {{
          "title": "<Recipe Title>",
          "calories": 0,
          "protein_g": 0,
          "fat_g": 0,
          "carbs_g": 0,
          "ingredients": ["<ingredient 1>", "<ingredient 2>", "..."],
          "units": ["<unit 1>", "<unit 2>", "..."],
          "quantities": ["<quantity 1>", "<quantity 2>", "..."],
          "instructions": "<full instructions text>"
        }},
        "dinner": {{
          "title": "<Recipe Title>",
          "calories": 0,
          "protein_g": 0,
          "fat_g": 0,
          "carbs_g": 0,
          "ingredients": ["<ingredient 1>", "<ingredient 2>", "..."],
          "units": ["<unit 1>", "<unit 2>", "..."],
          "quantities": ["<quantity 1>", "<quantity 2>", "..."],
          "instructions": "<full instructions text>"
        }}
      }}
    }}
  ]
}}
"""
)

In [ ]:
combined_rag_chain = (
    {
        "context": lambda _: context,
        "question": RunnablePassthrough(),  # user question for LLM
        "preferences": RunnablePassthrough(),  # user preference text
        "daily_cal": lambda _: DAILY_KCAL,
        "daily_protein": lambda _: DAILY_PROTEIN,
        "daily_fat": lambda _: DAILY_FAT,
        "daily_carbs": lambda _: DAILY_CARBS,
    }
    | combined_daily_prompt
    | cohere_chat_model
    | StrOutputParser()
)


In [ ]:
# @markdown We are reaching our output token limit, so we only get meals for part of the week...
# preferences = "avoid any spicy dishes"
preferences = results.get("original_request").get("preferences")
additional_preferences = preferences + ". Avoid spicy food and meals with mushrooms"
question = "Create a balanced 7-day meal plan for muscle gain."

result = combined_rag_chain.invoke({"question": question, "preferences": preferences})
print(result)

```json
{
  "days": [
    {
      "day": 1,
      "meals": {
        "breakfast": {
          "title": "Garden Frittata",
          "calories": 627.45,
          "protein_g": 46.525,
          "fat_g": 19.8225,
          "carbs_g": 35.73,
          "ingredients": [
            "egg substitute, powder",
            "onions, raw",
            "asparagus, raw",
            "broccoli, raw",
            "cheese, cheddar",
            "mushrooms, white, raw",
            "salt, table",
            "spices, pepper, black",
            "spices, curry powder",
            "spices, pepper, red or cayenne"
          ],
          "units": [
            "cup",
            "cup",
            "cup",
            "cup",
            "cup",
            "cup",
            "teaspoon",
            "teaspoon",
            "teaspoon",
            "teaspoon"
          ],
          "quantities": [
            "2",
            "1",
            "1",
            "1",
            "1",
            "12",
            

In [ ]:
# @title Tell LLM to only provide recipe IDs, then we match on them after
context = build_llm_context(results.get("candidate_recipes"))

combined_daily_prompt = PromptTemplate(
    input_variables=[
        "context",
        "question",
        "preferences",
        "daily_cal",
        "daily_protein",
        "daily_fat",
        "daily_carbs",
    ],
    template="""
You are a nutrition assistant helping to build a 7-day meal plan.
Each day should have 3 meals: Breakfast, Lunch, and Dinner.

Your goal is to select meals so that the **daily totals** are as close as possible to the targets below:

- Calories: {daily_cal} kcal
- Protein: {daily_protein} g
- Fat: {daily_fat} g
- Carbs: {daily_carbs} g

User preferences to consider (e.g. dislikes, dietary restrictions, flavor preferences):
{preferences}

Here is a list of candidate meals with macros, flavors, and instructions:
---
{context}
---

Question:
{question}

Return the meal plan strictly as valid JSON using this structure including the recipe_id values of the relevant recipes from the candidate meals. Do not make up recipe ids. Only use ids listed in the recipe_id field in the candidates:
{{
  "days": [
    {{
      "day": 1,
      "meals": {{
        "breakfast": {{
          "title": "<Recipe Title>",
          "calories": 0,
          "protein_g": 0,
          "fat_g": 0,
          "carbs_g": 0,
          "recipe_id": "<recipe_id>"
        }},
        "lunch": {{
          "title": "<Recipe Title>",
          "calories": 0,
          "protein_g": 0,
          "fat_g": 0,
          "carbs_g": 0,
          "recipe_id": "<recipe_id>"
        }},
        "dinner": {{
          "title": "<Recipe Title>",
          "calories": 0,
          "protein_g": 0,
          "fat_g": 0,
          "carbs_g": 0,
          "recipe_id": "<recipe_id>"
        }}
      }}
    }}
  ]
}}
"""
)

In [ ]:
combined_rag_chain = (
    {
        "context": lambda _: context,
        "question": RunnablePassthrough(),  # user question for LLM
        "preferences": RunnablePassthrough(),  # user preference text
        "daily_cal": lambda _: DAILY_KCAL,
        "daily_protein": lambda _: DAILY_PROTEIN,
        "daily_fat": lambda _: DAILY_FAT,
        "daily_carbs": lambda _: DAILY_CARBS,
    }
    | combined_daily_prompt
    | cohere_chat_model
    | StrOutputParser()
)

In [ ]:
# preferences = "avoid any spicy dishes"
preferences = results.get("original_request").get("preferences")
preferences = preferences + ". Avoid any spicy dishes." #additional preferences
question = "Create a balanced 7-day meal plan for muscle gain."

result = combined_rag_chain.invoke({"question": question, "preferences": preferences})
print(result)

```json
{
  "days": [
    {
      "day": 1,
      "meals": {
        "breakfast": {
          "title": "Garden Frittata",
          "calories": 627.45,
          "protein_g": 46.525,
          "fat_g": 19.8225,
          "carbs_g": 35.73,
          "recipe_id": "d1598d6fbc"
        },
        "lunch": {
          "title": "Chickpea, Pesto & Red Onion Salad",
          "calories": 672.325,
          "protein_g": 24.251125,
          "fat_g": 11.092875,
          "carbs_g": 71.251375,
          "recipe_id": "c4e1ad006c"
        },
        "dinner": {
          "title": "Vegan Soy, Lentil, & Veggie Burger Sliders Using Tvp",
          "calories": 889.609375,
          "protein_g": 46.81171875,
          "fat_g": 21.841796875,
          "carbs_g": 15.588984375,
          "recipe_id": "266444cabe"
        }
      }
    },
    {
      "day": 2,
      "meals": {
        "breakfast": {
          "title": "Garden Frittata",
          "calories": 627.45,
          "protein_g": 46.525,
          

In [ ]:
# Check to see if the recipe_ids are valid
recipe_id_list = [a["recipe_id"] for a in results.get("candidate_recipes")]

# Trim off the ```json ``` from the result
result_trim = result.split("```json")[1].split("```")[0]
llm_results = json.loads(result_trim)

for day in llm_results.get("days"):
  for meal_key,meal in day.get("meals").items():
    if meal.get("recipe_id") not in recipe_id_list:
      print(f"ERROR: nvalid recipe_id: {meal.get('recipe_id')} on day {day.get("day")} {meal_key}")
    else:
      recipe_index = recipe_id_list.index(meal.get("recipe_id"))
      # results.get("candidate_recipes")[recipe_index].get("recipe_id")
      print(f"Valid recipe_id: {meal.get('recipe_id')} on day {day.get("day")} {meal_key} index {recipe_index}")
      print(results.get("candidate_recipes")[recipe_index].get("recipe_id"))


Valid recipe_id: d1598d6fbc on day 1 breakfast index 0
d1598d6fbc
Valid recipe_id: c4e1ad006c on day 1 lunch index 14
c4e1ad006c
Valid recipe_id: 266444cabe on day 1 dinner index 16
266444cabe
Valid recipe_id: d1598d6fbc on day 2 breakfast index 0
d1598d6fbc
Valid recipe_id: be36d4b236 on day 2 lunch index 15
be36d4b236
Valid recipe_id: 1d66c87df7 on day 2 dinner index 29
1d66c87df7
Valid recipe_id: d1598d6fbc on day 3 breakfast index 0
d1598d6fbc
Valid recipe_id: 86a1200554 on day 3 lunch index 22
86a1200554
Valid recipe_id: f00ec9f59a on day 3 dinner index 21
f00ec9f59a
Valid recipe_id: d1598d6fbc on day 4 breakfast index 0
d1598d6fbc
Valid recipe_id: c4e1ad006c on day 4 lunch index 14
c4e1ad006c
Valid recipe_id: 59b78badd7 on day 4 dinner index 37
59b78badd7
Valid recipe_id: d1598d6fbc on day 5 breakfast index 0
d1598d6fbc
Valid recipe_id: be36d4b236 on day 5 lunch index 15
be36d4b236
Valid recipe_id: 266444cabe on day 5 dinner index 16
266444cabe
Valid recipe_id: d1598d6fbc on day 

In [ ]:

# @title Construct recipe list based on IDs

import copy

# Start with an empty dict
meal_plan = {
    "daily_plans": [],
}

target_calories = results.get("original_request").get("target_calories")
target_protein = results.get("original_request").get("target_protein")
target_fat = results.get("original_request").get("target_fat")
target_carbs = results.get("original_request").get("target_carbs")
for day in llm_results.get("days"):
  total_calories = 0
  total_protein = 0
  total_fat = 0
  total_carbs = 0
  day_meals = []
  for meal_key,meal in day.get("meals").items():
    # Find recipe by recipe_id in results
    # Find the index in recipe_id_list
    recipe_index = recipe_id_list.index(meal.get("recipe_id"))
    new_meal = results.get("candidate_recipes")[recipe_index]
    if new_meal.get("recipe_id") != meal.get("recipe_id"):
      print(f"Invalid recipe_id: {meal.get('recipe_id')} on day {day.get('day')} {meal_key}")
    day_meals.append(new_meal)
    total_calories += new_meal.get("calories")
    total_protein += new_meal.get("protein_g")
    total_fat += new_meal.get("fat_g")
    total_carbs += new_meal.get("carbs_g")
  day = {
      "day": day.get("day"),
      "meals": day_meals,
      "total_calories": total_calories,
      "total_protein": total_protein,
      "total_fat": total_fat,
      "total_carbs": total_carbs,
      "target_calories": target_calories,
      "target_protein": target_protein,
      "target_fat": target_fat,
      "target_carbs": target_carbs
  }
  meal_plan["daily_plans"].append(day)







In [ ]:
# @markdown Show the number of days returned
len(meal_plan.get("daily_plans"))

7

In [ ]:
# @markdown show just the first day
meal_plan.get("daily_plans")[0]

{'day': 1,
 'meals': [{'title': 'Garden Frittata',
   'calories': 627.45,
   'protein_g': 46.525,
   'fat_g': 19.8225,
   'carbs_g': 35.73,
   'description': 'A versatile, egg-based breakfast dish with vegetables and cheese, baked in a square pan; great for using up leftover ingredients.',
   'instructions': 'Pre heat oven to 400.\nMix all ingredients in a large mixing bowl.\nSpray an 8x8-inch baking dish with a nonstick cooking spray and pour all ingredients into the dish.\nBake for 30-45 minutes depending on your oven.\nWhen done let the frittata sit for 15 minutes (to set) and then slice and serve.\nUse your imagination and use different vegetables or whatever you have on hand.',
   'ingredients': ['egg substitute, powder',
    'onions, raw',
    'asparagus, raw',
    'broccoli, raw',
    'cheese, cheddar',
    'mushrooms, white, raw',
    'salt, table',
    'spices, pepper, black',
    'spices, curry powder',
    'spices, pepper, red or cayenne'],
   'quantities': ['2', '12', '1', 

LLM GENERATION WITH IDs

In [13]:
def build_llm_context(meal_candidates: list[dict], max_context: int = 80):
    """
    Builds a compact context for the LLM using only metadata fields.
    Each recipe includes: ID, title, macros, and a short description.
    """
    selected = meal_candidates[:max_context]
    context_lines = []
    for idx, recipe in enumerate(selected):
        rid = recipe.get("recipe_id", idx)
        title = recipe.get("title", "Unknown Recipe")
        cal = round(recipe.get("calories", 0), 1)
        protein = round(recipe.get("protein_g", 0), 1)
        fat = round(recipe.get("fat_g", 0), 1)
        carbs = round(recipe.get("carbs_g", 0), 1)
        desc = recipe.get("description", "")[:150]
        context_lines.append(
            f"ID: {rid} | Title: {title} | {cal} kcal | P:{protein}g F:{fat}g C:{carbs}g | {desc}"
        )
    return "\n".join(context_lines)

In [23]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("""
You are a meal planner AI assistant. Your job is to choose recipes for a 7-day plan based on calorie and macro goals.

USER PREFERENCES:
{preferences}

EXCLUSIONS:
{exclusions}

DIETARY FLAGS:
{dietary}

DAILY MACRO TARGETS:
Calories: {daily_calories}
Protein: {daily_protein}g
Fat: {daily_fat}g
Carbs: {daily_carbs}g

Below are candidate recipes with IDs and macros:

{context}

---

Please choose a combination of recipes for a 7-day plan:
- 1 breakfast, 1 lunch, and 1 dinner per day (21 meals total)
- Try to balance the total macros close to the daily targets.
- Respect user preferences and exclusions.
- Respond ONLY with a JSON list in this format:

{{
  "day_1": {{"breakfast": "<recipe_id>", "lunch": "<recipe_id>", "dinner": "<recipe_id>"}},
  "day_2": {{"breakfast": "<recipe_id>", "lunch": "<recipe_id>", "dinner": "<recipe_id>"}},
  "day_3": {{"breakfast": "<recipe_id>", "lunch": "<recipe_id>", "dinner": "<recipe_id>"}},
  "day_4": {{"breakfast": "<recipe_id>", "lunch": "<recipe_id>", "dinner": "<recipe_id>"}},
  "day_5": {{"breakfast": "<recipe_id>", "lunch": "<recipe_id>", "dinner": "<recipe_id>"}},
  "day_6": {{"breakfast": "<recipe_id>", "lunch": "<recipe_id>", "dinner": "<recipe_id>"}},
  "day_7": {{"breakfast": "<recipe_id>", "lunch": "<recipe_id>", "dinner": "<recipe_id>"}}
}}

Do NOT include any additional text or commentary.
""")

In [24]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


daily_macros = results.get("original_request")
context = build_llm_context(results.get("candidate_recipes"))

llm_chain = (
    {
        "preferences": RunnablePassthrough(),
        "exclusions": RunnablePassthrough(),
        "dietary": RunnablePassthrough(),
        "daily_calories": RunnableLambda(lambda _: daily_macros["target_calories"]),
        "daily_protein": RunnableLambda(lambda _: daily_macros["target_protein"]),
        "daily_fat": RunnableLambda(lambda _: daily_macros["target_fat"]),
        "daily_carbs": RunnableLambda(lambda _: daily_macros["target_carbs"]),
        "context": RunnableLambda(lambda _: context)
    }
    | prompt_template
    | cohere_chat_model
    | StrOutputParser()
)

In [25]:
output = llm_chain.invoke({
    "preferences": preferences,
    "exclusions": exclusions,
    "dietary": ", ".join(dietary)
})

print("=== SELECTED RECIPE IDs ===")
print(output)

=== SELECTED RECIPE IDs ===
```json
{
  "day_1": {
    "breakfast": "d1598d6fbc",
    "lunch": "c4e1ad006c",
    "dinner": "f00ec9f59a"
  },
  "day_2": {
    "breakfast": "d1598d6fbc",
    "lunch": "be36d4b236",
    "dinner": "266444cabe"
  },
  "day_3": {
    "breakfast": "d1598d6fbc",
    "lunch": "86a1200554",
    "dinner": "059802d73c"
  },
  "day_4": {
    "breakfast": "d1598d6fbc",
    "lunch": "31c2c820fd",
    "dinner": "59b78badd7"
  },
  "day_5": {
    "breakfast": "d1598d6fbc",
    "lunch": "f681d12b12",
    "dinner": "1d66c87df7"
  },
  "day_6": {
    "breakfast": "d1598d6fbc",
    "lunch": "88eba8e160",
    "dinner": "266444cabe"
  },
  "day_7": {
    "breakfast": "d1598d6fbc",
    "lunch": "be36d4b236",
    "dinner": "f00ec9f59a"
  }
}
```


In [32]:
from pprint import pprint
result_trim = output.split("```json")[1].split("```")[0]
llm_results = json.loads(result_trim)
candidate_recipes = results.get("candidate_recipes")
recipe_lookup = {r["recipe_id"]: r for r in candidate_recipes}
full_meal_plan = {"days": []}

for day_num in range(1, 8):  # 1 through 7
    day_key = f"day_{day_num}"
    day_plan_ids = llm_results.get(day_key)

    day_meals = {}
    total_calories = 0
    total_protein = 0
    total_fat = 0
    total_carbs = 0

    for meal_type, recipe_id in day_plan_ids.items():
        recipe = recipe_lookup.get(recipe_id)
        if not recipe:
            raise ValueError(f"Invalid recipe_id {recipe_id} for {meal_type} on {day_key}")

        day_meals[meal_type] = recipe

        # Accumulate daily totals
        total_calories += recipe.get("calories", 0)
        total_protein += recipe.get("protein_g", 0)
        total_fat += recipe.get("fat_g", 0)
        total_carbs += recipe.get("carbs_g", 0)

    full_meal_plan["days"].append({
        "day": day_num,
        "meals": day_meals,
        "total_calories": total_calories,
        "total_protein": total_protein,
        "total_fat": total_fat,
        "total_carbs": total_carbs,
        "target_calories": target_calories,
        "target_protein": target_protein,
        "target_fat": target_fat,
        "target_carbs": target_carbs
    })
pprint(full_meal_plan)

{'days': [{'day': 1,
           'meals': {'breakfast': {'calories': 627.45,
                                   'carbs_g': 35.73,
                                   'description': 'A versatile, egg-based '
                                                  'breakfast dish with '
                                                  'vegetables and cheese, '
                                                  'baked in a square pan; '
                                                  'great for using up leftover '
                                                  'ingredients.',
                                   'fat_g': 19.8225,
                                   'ingredients': ['egg substitute, powder',
                                                   'onions, raw',
                                                   'asparagus, raw',
                                                   'broccoli, raw',
                                                   'cheese, cheddar',
                    